In [1]:
import numpy as np 
import random
import cv2 
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

import pandas as pd
import time

In [2]:
def load(paths, verbose=-1):
    '''expects images for each class in seperate dir, 
    e.g all digits in 0 class in the directory named 0 '''
    data = list()
    labels = list()
    # loop over the input images
    for (i, imgpath) in enumerate(paths):
        # load the image and extract the class labels
        im_gray = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE) # the image is read as a gray scale
        image = np.array(im_gray).flatten() # the image is flattened #Return a copy of the array collapsed into one dimension.
        label = imgpath.split(os.path.sep)[-2] # for obtain the class label 
        # scale the image to [0, 1] and add to list
        data.append(image/255) # scale the image to [0, 1]
        labels.append(label)
        # show an update every `verbose` images
        if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
            print("[INFO] processed {}/{}".format(i + 1, len(paths)))
    # return a tuple of the data and labels
    return data, labels


In [3]:
#declare path to your mnist data folder
img_path = 'D:/Licenta/Proiect/dataSet'

In [4]:
#get the path list using the path object
image_paths = list(paths.list_images(img_path))

In [5]:
#apply our function
image_list, label_list = load(image_paths, verbose=10000)

[INFO] processed 10000/42000
[INFO] processed 20000/42000
[INFO] processed 30000/42000
[INFO] processed 40000/42000


In [6]:
# converteste etichetele claselor în format binar.
lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)


In [7]:
#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(image_list, 
                                                    label_list, 
                                                    test_size=0.1, 
                                                    random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)



In [8]:
def create_clientsIID(image_list, label_list, num_clients=1, initial='clients'):
    '''
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1       
    '''
    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1)
                    for i in range(num_clients)]
    #randomize the data
    data = list(zip(image_list, label_list))
    random.seed(10)
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]
    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))}

In [9]:
clients= create_clientsIID(X_train, y_train, num_clients=10, initial='client')

for client_name, client_data in clients.items():
    num_images = len(client_data)  
    client_label = client_data[0][1]  
    print(f" {client_name} : {num_images} , {client_label}.")
    


 client_1 : 3780 , [0 0 0 0 0 0 0 1 0 0].
 client_2 : 3780 , [0 0 1 0 0 0 0 0 0 0].
 client_3 : 3780 , [1 0 0 0 0 0 0 0 0 0].
 client_4 : 3780 , [0 0 0 0 0 0 1 0 0 0].
 client_5 : 3780 , [0 0 0 0 0 0 0 0 1 0].
 client_6 : 3780 , [0 0 0 0 0 0 1 0 0 0].
 client_7 : 3780 , [0 0 0 1 0 0 0 0 0 0].
 client_8 : 3780 , [1 0 0 0 0 0 0 0 0 0].
 client_9 : 3780 , [0 0 0 0 0 0 1 0 0 0].
 client_10 : 3780 , [1 0 0 0 0 0 0 0 0 0].


In [10]:
def flip_labels(clients, flip_fraction, clients_to_flip):
    '''
    flip a fraction of labels for selected clients 
      clients (dict): Dictionary containing client data 
      flip_fraction (float): Fraction of labels to flip
      clients_to_flip (list): List of client names for which labels will be flipped.
    '''
    for client_name in clients_to_flip:
        data = clients[client_name]  
        flipped_count = int(len(data) * flip_fraction)  
        flip_indices = random.sample(range(len(data)), flipped_count)  

        for i in flip_indices:
            original_label = data[i][1] 
            new_label = random.randint(0, 9)  
            while new_label == np.argmax(original_label):
                new_label = random.randint(0, 9)

            flipped_label = np.zeros(original_label.shape)
            flipped_label[new_label] = 1
            data[i] = (data[i][0], flipped_label)

    return clients

clients_to_flip = random.sample(list(clients.keys()), 8)
flipped_clients= flip_labels(clients, 1, clients_to_flip)

In [11]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [12]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [13]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(256, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(128))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

In [14]:
learning_rate = 0.01
comms_round = 100

loss='categorical_crossentropy'
metrics = ['accuracy']

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000, 
    decay_rate=0.9) 

optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
# Create the optimizer with the learning rate schedule
optimizer = SGD(learning_rate=lr_schedule, momentum=0.9)

In [15]:
#global_count = total number of data used for all clients
#local_count = number of data for one client
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clients
    global_count = sum([tf.data.experimental.cardinality(
                   clients_trn_data[client_name]).numpy() 
                   for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(
                  clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final


def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights.
    The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    global performanceExport
    
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    
    new_performanceExport = pd.DataFrame({'Model':'GLOBAL', 'Com_Round': int(comm_round), 'accuracy': float(acc * 100), 'loss': float(loss)}, index=[0])
    performanceExport = pd.concat([performanceExport, new_performanceExport])
    
    path = f'D:/Licenta/Proiect/exportData/export_dataframe.xlsx'
    performanceExport.to_excel (path, index = False, header=True)
   
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss



In [16]:
performanceExport = pd.DataFrame(columns = ['Model', 'Com_Round', 'accuracy'])
accuracy = pd.DataFrame(columns = ['Model', 'Com_Round', 'accuracy'])

def test_model(X_test, Y_test,  model, comm_round):
    global performanceExport
    
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    
    new_performanceExport = pd.DataFrame({'Model':'GLOBAL', 'Com_Round': int(comm_round), 'accuracy': float(acc * 100), 'loss': float(loss)}, index=[0])
    performanceExport = pd.concat([performanceExport, new_performanceExport])
    
    path = f'D:/Licenta/Proiect/exportData/export_dataframe.xlsx'
    performanceExport.to_excel (path, index = False, header=True)
   
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

#print the performances from each client using test data
def printLocalPerfm(X_test, y_test, model, comm_round, clientID):
    global performanceExport
    global accuracy
    
    #list for local accuracy and loss (for each model from client)
    global local_acc_list 
    local_acc_list = []
    global local_loss_list
    local_loss_list = []
    
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test) 
    loss = cce(Y_test, logits) 
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    local_acc_list.append(loss)
    local_loss_list.append(acc)

    new_acc = pd.DataFrame({'Model': f'model_{int(clientID)}', 'Com_Round': int(comm_round), 'accuracy': float(acc * 100)}, index=[0])
    accuracy = pd.concat([accuracy, new_acc])
    
  
    new_performanceExport = pd.DataFrame({'Model': f'model_{int(clientID)}', 'Com_Round': int(comm_round), 'accuracy': float(acc * 100), 'loss': float(loss)}, index=[0])
    performanceExport = pd.concat([performanceExport, new_performanceExport])
   
    path = f'D:/Licenta/Proiect/exportData/export_dataframe.xlsx'
    performanceExport.to_excel (path, index = False, header=True) 
    
    print('comm_round: {} | local_acc: {:.3%} | local_loss: {}'.format(comm_round, acc, loss))
    return acc, loss


In [ ]:
#Metoda 3 
#initialize global model
start_time = time.time()
smlp_global = SimpleMLP()
global_model = smlp_global.build(784, 10)
#commence global training loop
for comm_round in range(comms_round):        
    # get the global model's weights
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()
    client_names= list(clients_batched.keys())
    random.seed(448)
    random.shuffle(client_names)
    clientID = 0
    local_accuracy_list = []
    local_weight_list=[]
    
    #loop through each client and create new local model
    for client in client_names:
        print(client)
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784, 10)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        for(X_test, Y_test) in test_batched:
            local_acc, local_loss = printLocalPerfm(X_test, Y_test, 
                                    local_model, comm_round,clientID)
        
        local_accuracy_list.append(local_acc)
        local_weight_list.append(local_model.get_weights())
        
        #clear session to free memory after each communication round
        K.clear_session()
        clientID += 1

    # calculate sum of local accuracies
    sum_local_acc = sum(local_accuracy_list)
    # calculate the scaling factors for each client's weights
    scaling_factors = [acc / sum_local_acc for acc in local_accuracy_list]
    #scale the local weights
    scaled_local_weight_list = [[scaling_factors[i] * weight for weight
                                 in local_weight_list[i]] for i in 
                                 range(len(local_weight_list))]
    
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    #update global model 
    global_model.set_weights(average_weights)
   
    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

            
end_time = time.time()
training_time = end_time - start_time
print("Timpul de antrenare:", training_time, "secunde")
